In [1]:
import pickle
import pandas as pd
import numpy as np
import re
import os
import unicodedata

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 500)

In [2]:
# number of col_names in list must be equal number of rows df
def transform_columns_to_rows(df: pd.DataFrame, col_names: list):
    pd_series = pd.DataFrame()
    for i, col in zip(range(df.shape[0]), col_names):
        pd_series[col] = df.stack()[i]
    return pd_series 

In [3]:
all_events_games = list()
for file in os.listdir('pickle_files'):
    if re.search('-Copy1', file):
        with open('pickle_files/' + file, 'rb') as f:
            events_games = pickle.load(f)
            all_events_games.append(events_games)

In [5]:
len(all_events_games)

13

In [6]:
with open('pickle_files/top_ligues_start_1_11920200-Copy1', 'rb') as f:
    events_games = pickle.load(f)

In [8]:
df_to_transform = pd.DataFrame(events_games)

In [9]:
df = transform_columns_to_rows(df_to_transform, ['event_mins', 'event_hts_ats', 'stats_dict', 'city_country', 'viewers', 'weath_temp', 'bet_coefs'])

In [10]:
df.reset_index(inplace=True)
df.rename(columns={'index':'game_id'}, inplace=True)

In [11]:
df.shape

(7995, 8)

#### Calculate fist minute's goal

In [12]:
sr_goals = df[['event_mins', 'event_hts_ats']].apply(lambda x: {k:v for k, v in zip(x['event_mins'], x['event_hts_ats']) if re.findall('goal', v)}, axis=1)   

In [13]:
df['fg_minute'] = sr_goals.map(lambda x: next(iter(x.keys())) if x != {} else None)

#### Create cards columns

In [14]:
df['y_cards'] = df.stats_dict.map(lambda x: [int(i[1]) + int(i[2]) for i in x if re.findall('Yellow cards', next(iter(i)))])
df.y_cards = df.y_cards.map(lambda x: ''.join(map(str, x)))

In [15]:
df['r_cards'] = df.stats_dict.map(lambda x: [int(i[1]) + int(i[2]) for i in x if re.findall('Red cards', next(iter(i)))])
df.r_cards = df.r_cards.map(lambda x: ''.join(map(str, x)))

In [16]:
df.r_cards.value_counts()

0    6410
1    1204
      191
2     168
3      18
4       3
6       1
Name: r_cards, dtype: int64

In [17]:
df.y_cards  = df[['event_hts_ats', 'y_cards']].apply(lambda x: x['event_hts_ats'].count('yellowcard') if x['y_cards'] == '' else x['y_cards'], axis=1) 
df.r_cards  = df[['event_hts_ats', 'r_cards']].apply(lambda x: x['event_hts_ats'].count('redcard') if x['r_cards'] == '' else x['r_cards'], axis=1) 

In [18]:
df.y_cards = df.y_cards.astype(int)
df.r_cards = df.r_cards.astype(int)

In [19]:
df.r_cards.value_counts()

0    6584
1    1219
2     170
3      18
4       3
6       1
Name: r_cards, dtype: int64

In [20]:
df['card_coef'] = df.y_cards + df.r_cards*2

#### Calculate 'mispen' and 'owngoal' in event_hts_ats

In [21]:
df['mispen'] = df.event_hts_ats.map(lambda x: True if any(i == 'mispen' for i in x) else False)
df['owngoal'] = df.event_hts_ats.map(lambda x: True if any(i == 'owngoal' for i in x) else False)

In [22]:
df.mispen.value_counts()

False    7480
True      515
Name: mispen, dtype: int64

In [23]:
df.owngoal.value_counts()

False    7454
True      541
Name: owngoal, dtype: int64

#### Calculate bets coefficients

In [24]:
sr_bets = df.bet_coefs.map(lambda x:{i:v for i, v in enumerate(x[1])})                  

In [25]:
df_bets = pd.DataFrame(sr_bets)
df_bets.head(1)

,bet_coefs
0,"{0: '2.49', 1: '3.66', 2: '2.80', 3: '3.20', 4: '1.37'}"


In [26]:
df_bets = pd.DataFrame(df_bets['bet_coefs'].values.tolist(), index=df_bets.index)
df_bets.columns = ['1', 'X', '2', 'TM', 'TB']

In [29]:
df_bets.head(3)

,1,X,2,TM,TB
0,2.49,3.66,2.80,3.20,1.37
1,1.71,4.32,4.82,2.32,1.58
2,2.32,3.09,3.60,1.36,3.24


In [30]:
df[df_bets.columns] = df_bets

In [33]:
df.city_country.value_counts().head(5)

                          593
London, United Kingdom    262
Sao Paulo, Brazil         244
Rio de Janeiro, Brazil    195
Belo Horizonte, Brazil    149
Name: city_country, dtype: int64

#### Create country and city columns

In [34]:
df['city'] = df.city_country.map(lambda x: x.split(',')[0] if len(x.split(',')) > 1 else None)
df['country']  = df.city_country.map(lambda x: x.split(',')[1] if len(x.split(',')) > 1 else (x.split(',')[0] if x is not  None else None)) 

#### Merge df with games statistic with df_all_mdf_all_matches

In [35]:
col_names = ['game_id', 'fg_minute', 'y_cards', 'r_cards' ,'card_coef', 'mispen', 'owngoal', '1', 'X', '2', 'TM', 'TB', 'city', 'country']
df = df[col_names].copy()

In [36]:
with open('pickle_files/df_all_matches', 'rb') as f:
    df_all_matches = pickle.load(f)

In [37]:
df_matches = df_all_matches[df_all_matches.game_id.isin(df.game_id)]

In [38]:
df_matches.shape

(7995, 8)

In [39]:
df_matches = df_matches.merge(df, how='left', left_on='game_id', right_on='game_id').copy()

In [60]:
df_matches.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,game_status,fg_minute,y_cards,r_cards,card_coef,mispen,owngoal,1,X,2,TM,TB,city,country,home_team,guest_team
0,14898157,1. Bundesliga,17,300,05.10.2019 16:30,Paderborn - Mainz,1:2,Finished,8,5,0,5,True,False,2.49,3.66,2.80,3.20,1.37,Paderborn,Germany,Paderborn,Mainz
1,14898156,1. Bundesliga,17,300,05.10.2019 19:30,Schalke - Köln,1:1,Finished,72,8,0,8,False,False,1.71,4.32,4.82,2.32,1.58,Gelsenkirchen,Germany,Schalke,Köln
2,14900649,Premier Liga,13,300,05.10.2019 11:30,Ufa - Ahmat,0:1,Finished,9,4,0,4,False,False,2.32,3.09,3.60,1.36,3.24,Ufa,Russia,Ufa,Ahmat


#### Create home_team and guest_team columns

In [58]:
df_matches['home_team'] = df_matches.game_title.map(lambda x: x.split('-')[0].rstrip())
df_matches['guest_team']  = df_matches.game_title.map(lambda x: x.split('-')[1].rstrip()) 

#### Load df_teams_data for geting teams city and country in empty cells

In [190]:
with open('pickle_files/df_teams_data', 'rb') as f:
    df_teams_data = pickle.load(f)

In [192]:
df_teams_data.head(3)

,club,club_country,club_city
0,ABC,Brazil,Natal
1,ACS Poli Timisoara,Romania,Timisoara
2,ADO Den Haag,Netherlands,Den Haag


##### Change no utf-8 symbols

In [193]:
def find_no_utf_symbols(sr: pd.Series):
    find_symols = r'[^.a-zA-Z0-9)(&"\s>;<=,-/}{\']'
    rare_symbols = sr.map(lambda x: ''.join(re.findall(find_symols, str(x))))
    return rare_symbols

In [194]:
no_utf_symbols = find_no_utf_symbols(df_matches.home_team)
str(set(no_utf_symbols))

"{''}"

In [195]:
def str_decode(str_obj: str):
    normalized = unicodedata.normalize('NFD', str_obj)
    decode_str = u"".join([c for c in normalized if not unicodedata.combining(c)])
    return decode_str

In [196]:
df_matches.home_team = df_matches.home_team.map(lambda x: str_decode(str(x))) 
df_matches.guest_team = df_matches.guest_team.map(lambda x: str_decode(str(x))) 

In [223]:
df_matches.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,game_status,fg_minute,y_cards,r_cards,card_coef,mispen,owngoal,1,X,2,TM,TB,city,country,home_team,guest_team
0,14898157,1. Bundesliga,17,300,05.10.2019 16:30,Paderborn - Mainz,1:2,Finished,8,5,0,5,True,False,2.49,3.66,2.80,3.20,1.37,Paderborn,Germany,Paderborn,Mainz
1,14898156,1. Bundesliga,17,300,05.10.2019 19:30,Schalke - Köln,1:1,Finished,72,8,0,8,False,False,1.71,4.32,4.82,2.32,1.58,Gelsenkirchen,Germany,Schalke,Koln
2,14900649,Premier Liga,13,300,05.10.2019 11:30,Ufa - Ahmat,0:1,Finished,9,4,0,4,False,False,2.32,3.09,3.60,1.36,3.24,Ufa,Russia,Ufa,Ahmat


In [198]:
len(df_matches[df_matches.city.isna()].home_team.unique())

32

In [199]:
miss_city_teams = df_matches[df_matches.city.isna()].home_team.unique()
miss_city_teams

array(['Sao Bento', 'Parana', 'Vila Nova', 'Guarani Campinas', 'Oeste',
       'Brasil de Pelotas', 'Atletico Goianiense', 'Criciuma', 'Cuiaba',
       'Operario Ferroviario', 'Bragantino', 'CRB', 'Vitoria',
       'Botafogo SP', 'Nimes', 'Avai', 'Goias', 'Paysandu', 'Boa',
       'Juventude', 'Sampaio Correa', 'Figueirense', 'Ponte Preta',
       'Alagoano', 'Luverdense', 'ABC', 'Nautico', 'Ceara', 'Veres',
       'Shakhtar', 'Internacional', 'Santa Cruz'], dtype=object)

In [200]:
df_miss_city = pd.DataFrame()

for x in miss_city_teams:
    miss_city = df_teams_data[df_teams_data.club.str.contains(x, regex=False)]
    df_miss_city = pd.concat([df_miss_city, miss_city])

df_miss_city.reset_index(drop=True, inplace=True)

In [205]:
df_miss_city.head(3)

,club,club_country,club_city
0,Sao Bento,Brazil,Sorocaba
1,Athletico Paranaense,Brazil,Curitiba
2,Parana,Brazil,Curitiba


In [224]:
clubs_not_in_teams_data = np.setdiff1d(miss_city_teams, df_miss_city.club)
clubs_not_in_teams_data

array([], dtype=object)

In [209]:
df_matches = df_matches.merge(df_miss_city[['club', 'club_country', 'club_city']], how='left', left_on='home_team', right_on='club')

In [212]:
df_matches.club_city.value_counts().head()

Goiania          151
Campinas         102
Curitiba         100
Florianopolis    100
Maceio            81
Name: club_city, dtype: int64

In [218]:
df_matches.city = np.where(df_matches.city.isna(), df_matches.club_city, df_matches.city)
df_matches.country = np.where(df_matches.country.isna(), df_matches.club_country, df_matches.country)

In [220]:
df_matches[df_matches.city.isna()]

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,game_status,fg_minute,y_cards,r_cards,card_coef,mispen,owngoal,1,X,2,TM,TB,city,country,home_team,guest_team,club,club_country,club_city


In [221]:
df_matches[df_matches.country.isna()]

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,game_status,fg_minute,y_cards,r_cards,card_coef,mispen,owngoal,1,X,2,TM,TB,city,country,home_team,guest_team,club,club_country,club_city


In [222]:
df_matches.drop(columns=['club', 'club_country', 'club_city'], inplace=True)

### Get cities Longitude and Latitude

In [225]:
df_matches.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,game_status,fg_minute,y_cards,r_cards,card_coef,mispen,owngoal,1,X,2,TM,TB,city,country,home_team,guest_team
0,14898157,1. Bundesliga,17,300,05.10.2019 16:30,Paderborn - Mainz,1:2,Finished,8,5,0,5,True,False,2.49,3.66,2.80,3.20,1.37,Paderborn,Germany,Paderborn,Mainz
1,14898156,1. Bundesliga,17,300,05.10.2019 19:30,Schalke - Köln,1:1,Finished,72,8,0,8,False,False,1.71,4.32,4.82,2.32,1.58,Gelsenkirchen,Germany,Schalke,Koln
2,14900649,Premier Liga,13,300,05.10.2019 11:30,Ufa - Ahmat,0:1,Finished,9,4,0,4,False,False,2.32,3.09,3.60,1.36,3.24,Ufa,Russia,Ufa,Ahmat


In [232]:
# df_matches['location'] = df_matches[['city', 'country']].apply(lambda x: geolocator.geocode(x['city']+','+ x['country']), axis=1)

In [233]:
len(df_matches.city.unique())

190

In [227]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Your_Name")

city ="Agra"
country ="India"
loc = geolocator.geocode(city+','+ country)
lat = loc.latitude
long = loc.longitude

In [230]:
loc

Location(Agra, Uttar Pradesh, 280001, India, (27.1752554, 78.0098161, 0.0))